In [ ]:
import numpy as np
import pandas as pd
import torch
import cv2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import pydicom
import matplotlib.pyplot as plt
import colorsys
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# coding: utf-8
__author__ = 'ZFTurbo: https://kaggle.com/zfturbo'

# Custom by me

import numpy as np

def bb_intersection_over_union(A, B):
    xA = max(A[0], B[0])
    yA = max(A[1], B[1])
    xB = min(A[2], B[2])
    yB = min(A[3], B[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)

    if interArea == 0:
        return 0.0, 0.0

    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = (A[2] - A[0]) * (A[3] - A[1])
    boxBArea = (B[2] - B[0]) * (B[3] - B[1])

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return interArea, iou


def prefilter_boxes(boxes, scores, labels, weights, thr):
    # Create dict with boxes stored by its label
    new_boxes = dict()
    for t in range(len(boxes)):
        for j in range(len(boxes[t])):
            score = scores[t][j]
            if score < thr:
                continue
            label = int(labels[t][j])
            box_part = boxes[t][j]
            b = [int(label), float(score) * weights[t], float(box_part[0]), float(box_part[1]), float(box_part[2]), float(box_part[3])]
            if label not in new_boxes:
                new_boxes[label] = []
            new_boxes[label].append(b)

    # Sort each list in dict by score and transform it to numpy array
    for k in new_boxes:
        current_boxes = np.array(new_boxes[k])
        new_boxes[k] = current_boxes[current_boxes[:, 1].argsort()[::-1]]

    return new_boxes


def get_weighted_box(boxes, conf_type='avg'):
    """
    Create weighted box for set of boxes
    :param boxes: set of boxes to fuse 
    :param conf_type: type of confidence one of 'avg' or 'max'
    :return: weighted box
    """

    box = np.zeros(6, dtype=np.float32)
    conf = 0
    conf_list = []
    for b in boxes:
        box[2:] += (b[1] * b[2:])
        conf += b[1]
        conf_list.append(b[1])
    box[0] = boxes[0][0]
    if conf_type == 'avg':
        box[1] = conf / len(boxes)
    elif conf_type == 'max':
        box[1] = np.array(conf_list).max()
    box[2:] /= conf
    return box


def find_matching_box(boxes_list, new_box, match_iou):
    best_iou = match_iou
    best_index = -1
    for i in range(len(boxes_list)):
        box = boxes_list[i]
        if box[0] != new_box[0]:
            continue
        interArea, iou = bb_intersection_over_union(box[2:], new_box[2:])
        boxAArea = (box[2:][2] - box[2:][0]) * (box[2:][3] - box[2:][1])
        if iou > best_iou or interArea >= 0.9 * boxAArea:
            best_index = i
            best_iou = iou

    return best_index, best_iou


def weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=None, iou_thr=0.55, skip_box_thr=0.0, conf_type='avg', allows_overflow=False):
    """
    :param boxes_list: list of boxes predictions from each model, each box is 4 numbers.
    It has 3 dimensions (models_number, model_preds, 4)
    Order of boxes: x1, y1, x2, y2. We expect float normalized coordinates [0; 1]
    :param scores_list: list of scores for each model
    :param labels_list: list of labels for each model
    :param weights: list of weights for each model. Default: None, which means weight == 1 for each model
    :param iou_thr: IoU value for boxes to be a match
    :param skip_box_thr: exclude boxes with score lower than this variable
    :param conf_type: how to calculate confidence in weighted boxes. 'avg': average value, 'max': maximum value
    :param allows_overflow: false if we want confidence score not exceed 1.0

    :return: boxes: boxes coordinates (Order of boxes: x1, y1, x2, y2).
    :return: scores: confidence scores
    :return: labels: boxes labels
    """

    if weights is None:
        weights = np.ones(len(boxes_list))
    if len(weights) != len(boxes_list):
        print('Warning: incorrect number of weights {}. Must be: {}. Set weights equal to 1.'.format(len(weights), len(boxes_list)))
        weights = np.ones(len(boxes_list))
    weights = np.array(weights)

    if conf_type not in ['avg', 'max']:
        print('Unknown conf_type: {}. Must be "avg" or "max"'.format(conf_type))
        exit()

    filtered_boxes = prefilter_boxes(boxes_list, scores_list, labels_list, weights, skip_box_thr)
    if len(filtered_boxes) == 0:
        return np.zeros((0, 4)), np.zeros((0,)), np.zeros((0,))

    overall_boxes = []
    for label in filtered_boxes:
        boxes = filtered_boxes[label]
        new_boxes = []
        weighted_boxes = []

        # Clusterize boxes
        for j in range(0, len(boxes)):
            index, best_iou = find_matching_box(weighted_boxes, boxes[j], iou_thr)
            if index != -1:
                new_boxes[index].append(boxes[j])
                weighted_boxes[index] = get_weighted_box(new_boxes[index], conf_type)
            else:
                new_boxes.append([boxes[j].copy()])
                weighted_boxes.append(boxes[j].copy())

        # Rescale confidence based on number of models and boxes
        for i in range(len(new_boxes)):
            if not allows_overflow:
                weighted_boxes[i][1] = weighted_boxes[i][1] * min(weights.sum(), len(new_boxes[i])) / weights.sum()
            else:
                weighted_boxes[i][1] = weighted_boxes[i][1] * len(new_boxes[i]) / weights.sum()
        overall_boxes.append(np.array(weighted_boxes))

    overall_boxes = np.concatenate(overall_boxes, axis=0)
    overall_boxes = overall_boxes[overall_boxes[:, 1].argsort()[::-1]]
    boxes = overall_boxes[:, 2:]
    scores = overall_boxes[:, 1]
    labels = overall_boxes[:, 0]
    return boxes, scores, labels

In [ ]:
image_paths = r'../input/vinbigdata-chest-xray-abnormalities-detection/train'
df = pd.read_csv(r'../input/vinbigdata-chest-xray-abnormalities-detection/train.csv')
SIZE = 1024

df.drop(columns=["class_name"], inplace=True)

df = df[df.class_id != 14]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=101)
df_folds = df[['image_id']].copy()

df_folds.loc[:, 'bbox_count'] = 1
df_folds = df_folds.groupby('image_id').count()
df_folds.loc[:, 'object_count'] = df.groupby('image_id')['class_id'].nunique()

df_folds.loc[:, 'stratify_group'] = np.char.add(
    df_folds['object_count'].values.astype(str),
    df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str)
)

df_folds.loc[:, 'fold'] = 0
for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
    df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number

# example with fold 0
df_folds.reset_index(inplace=True)

valid_df = pd.merge(df, df_folds[df_folds['fold'] == 0], on='image_id')
train_df = pd.merge(df, df_folds[df_folds['fold'] != 0], on='image_id')

In [ ]:
def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=SIZE, width=SIZE, p=1.0),
            ToTensorV2(p=1.0),
        ],
        p=1.0,
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0,
            min_visibility=0,
            label_fields=['labels']
        )
    )


class DatasetRetriever(Dataset):

    def __init__(self, marking, image_dir=None, transforms=None):
        super().__init__()

        self.image_ids = marking["image_id"].unique()
        self.marking = marking
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image, boxes, labels = self.load_image_and_boxes(index)
        target = {'boxes': torch.from_numpy(boxes),
                  'labels': torch.from_numpy(labels),
                  'image_id': torch.tensor([index])}

        if self.transforms:
            sample = self.transforms(**{
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            })
            if len(sample['bboxes']) > 0:
                image = sample['image']
                target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                target['boxes'][:, [0, 1, 2, 3]] = target['boxes'][:, [1, 0, 3, 2]]
        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def load_image_and_boxes(self, index):

        image_id = self.image_ids[index]
        dicom = pydicom.dcmread(f"{self.image_dir}/{image_id}.dicom")
        image = dicom.pixel_array
        if "PhotometricInterpretation" in dicom:
            if dicom.PhotometricInterpretation == "MONOCHROME1":
                image = np.amax(image) - image

        image = np.stack([image, image, image])
        image = image.astype('float32')
        image = image - image.min()
        image = image / image.max()
        image = image.transpose(1, 2, 0)

        records = self.marking[self.marking['image_id'] == image_id]
        boxes = records[['x_min', 'y_min', 'x_max', 'y_max']].values

        records = self.marking[(self.marking['image_id'] == image_id)]
        records = records.reset_index(drop=True)
        labels = records["class_id"].values
        return image, boxes, np.array(labels, dtype=np.int)

In [ ]:
train_dataset = DatasetRetriever(marking=train_df, image_dir=image_paths, transforms=get_valid_transforms())

In [ ]:
def hsv2rgb(h, s, v):
    return tuple(round(i * 255) for i in colorsys.hsv_to_rgb(h, s, v))

In [ ]:
def visualize():
    mapping = {0: 'Aortic enlargement', 1: 'Atelectasis', 2: 'Calcification', 3: 'Cardiomegaly', 4: 'Consolidation',
               5: 'ILD', 6: 'Infiltration', 7: 'Lung Opacity', 8: 'Nodule/Mass', 9: 'Other lesion',
               10: 'Pleural effusion', 11: 'Pleural thickening', 12: 'Pneumothorax', 13: 'Pulmonary fibrosis'}

    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 1
    thickness = 4
    for i in range(30):
        image, target, image_ids = train_dataset[i]
        boxes = target['boxes'].cpu().numpy().astype(np.int64)
        labels = target['labels'].cpu().numpy().astype(np.int64)
        numpy_image = image.permute(1, 2, 0).cpu().numpy() * 255
        fig, ax = plt.subplots(1, 1, figsize=(16, 8))
        image_before = numpy_image.copy()
        for box, label in zip(boxes, labels):
            box = np.array(box, dtype=np.int)
            label = np.array(label, dtype=np.int)
            image_before = cv2.rectangle(image_before, (int(box[1]), int(box[0])), (int(box[3]), int(box[2])),
                                         hsv2rgb(int(label) / 14, 1, 1), 2)
            image_before = cv2.putText(image_before, mapping[int(label)], (box[1], box[0]), font, fontScale,
                                       hsv2rgb(int(label) / 14, 1, 1), thickness, cv2.LINE_AA)

        image_before = cv2.putText(image_before, 'BOXES_BEFORE', (100, 100), font, fontScale,
                                   hsv2rgb(1, 1, 1), thickness, cv2.LINE_AA)

        scores = [list(np.ones(labels.shape[0]))]
        boxes = [[box for box in boxes]]
        labels = [[label for label in labels]]
        boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, iou_thr=0.5)

        image_after = numpy_image.copy()
        for box, label in zip(boxes, labels):
            box = np.array(box, dtype=np.int)
            label = np.array(label, dtype=np.int)
            image_after = cv2.rectangle(image_after, (int(box[1]), int(box[0])), (int(box[3]), int(box[2])),
                                        hsv2rgb(int(label) / 14, 1, 1), 2)
            image_after = cv2.putText(image_after, mapping[int(label)], (box[1], box[0]), font, fontScale,
                                      hsv2rgb(int(label) / 14, 1, 1), thickness, cv2.LINE_AA)

        image_after = cv2.putText(image_after, 'BOXES_OPTIMIZED', (100, 100), font, fontScale,
                                  hsv2rgb(1, 1, 1), thickness, cv2.LINE_AA)

        image = cv2.hconcat([image_before, image_after])
        ax.imshow(image.astype(np.uint8))
        plt.show()

In [ ]:
if __name__ == '__main__':
    visualize()